In [1]:
import pandas as pd
import numpy as np
from lightgbm import  Booster
import lightgbm as lgb

from sklearn.metrics import mean_squared_error, r2_score

print(f"lightgbm version: {lgb.__version__}")

lightgbm version: 4.6.0


In [2]:

# Load the data
test_data = pd.read_csv('data/test.csv')


In [3]:
model2 = Booster(
    model_file='model.txt'
)

X_test = test_data.drop('target', axis=1)
y_test = test_data['target']
# Make predictions on test set
test_predictions = model2.predict(X_test)

# Evaluate the model
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
test_r2 = r2_score(y_test, test_predictions)

print()
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test R²: {test_r2:.4f}")




Test RMSE: 26.7448
Test R²: 0.9727
